In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,810 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main amd64 Packages [926 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:13 http://archive.ubuntu.com/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-10-24 05:32:05--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.77MB/s    in 0.5s    

2021-10-24 05:32:06 (1.77 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
#replace <your ...> with your database connection information
mode="append"
jdbc_url = "jdbc:postgresql://mypostgresdb.<>.rds.amazonaws.com:5432/my_data_base_db"
config = {"user":"root", "password": "<>", "driver":"org.postgresql.Driver"}

In [15]:
video_game_reviews = spark.read.jdbc(url=jdbc_url,table="video_game_vine_reviews", properties=config)
video_game_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R108YXX9X92XRQ|          5|            1|          1|   Y|
|R1EMOUJ76U86ZC|          5|            2|          3|   Y|
| RLHS2TBW58XX7|          4|            0|          0|   Y|
| RCC23D88Y2QDC|          5|            3|          3|   Y|
|R24PS9N276OXF4|          5|          241|        262|   Y|
|R1OKWU37KMFS8B|          5|            0|          0|   Y|
|R18EJDX9IX3JF3|          3|            0|          0|   Y|
|R21O8AAVY3M8KB|          5|            0|          0|   Y|
| RWW8JGLKWAT76|          2|            5|         11|   Y|
|R325H4RLI9LLM3|          3|            0|          3|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [6]:
software_reviews = spark.read.jdbc(url=jdbc_url, table="software_vine_reviews", properties=config)
software_reviews.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1RW6ARVYGN410|          4|            3|          5|   Y|
|R100Z9FFN9D73B|          5|            0|          2|   Y|
| RYMCEXWWQETDB|          4|            0|          0|   Y|
|R2XN3NK57JEJTQ|          5|            0|          0|   Y|
|R36QWUD2SMRLUO|          4|            0|          0|   Y|
| RJF642Q0WFZMZ|          2|            0|          0|   Y|
|R3LZ0FGVU40DSY|          4|            0|          0|   Y|
|R2YT6R26UBM17X|          3|            0|          0|   Y|
|R3J0ODKEVRU9A7|          4|            2|          4|   Y|
|R166YOVN38LSW1|          5|            0|          0|   Y|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [16]:
video_game_reviews.describe().show()

+-------+--------------+------------------+------------------+------------------+----+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes|vine|
+-------+--------------+------------------+------------------+------------------+----+
|  count|          4290|              4290|              4290|              4290|4290|
|   mean|          null|4.0748251748251745| 2.348717948717949|3.2783216783216784|null|
| stddev|          null| 0.918322230494836|12.580450532105258|13.769965761758156|null|
|    min|R10079FVQQF194|                 1|                 0|                 0|   Y|
|    max| RZXZZ98IW0F2G|                 5|               347|               362|   Y|
+-------+--------------+------------------+------------------+------------------+----+



In [8]:
software_reviews.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|
+-------+--------------+------------------+------------------+------------------+-----+
|  count|         10415|             10415|             10415|             10415|10415|
|   mean|          null| 3.839174267882861|3.1916466634661544| 3.871435429668747| null|
| stddev|          null|1.1056021641842058| 24.91389719042122|25.626546433058138| null|
|    min|R1000QJKXGZITK|                 1|                 0|                 0|    Y|
|    max| RZZ8LCK0XZBRA|                 5|              1231|              1247|    Y|
+-------+--------------+------------------+------------------+------------------+-----+



In [17]:
from pyspark.sql.functions import round
video_game_reviews_count = video_game_reviews.count()
video_game_helpful_vote = video_game_reviews.groupBy("helpful_votes").count()
video_game_helpful_vote = video_game_helpful_vote.orderBy(video_game_helpful_vote["count"].desc())
video_game_helpful_vote.withColumn('percentage', round(video_game_helpful_vote["count"]/video_game_reviews_count,4)).show()

+-------------+-----+----------+
|helpful_votes|count|percentage|
+-------------+-----+----------+
|            0| 2314|    0.5394|
|            1|  905|     0.211|
|            2|  370|    0.0862|
|            3|  201|    0.0469|
|            4|  136|    0.0317|
|            5|   69|    0.0161|
|            6|   51|    0.0119|
|            7|   40|    0.0093|
|            8|   21|    0.0049|
|           10|   18|    0.0042|
|            9|   16|    0.0037|
|           12|   14|    0.0033|
|           11|   11|    0.0026|
|           13|   10|    0.0023|
|           16|    9|    0.0021|
|           19|    7|    0.0016|
|           14|    6|    0.0014|
|           21|    6|    0.0014|
|           15|    5|    0.0012|
|           17|    5|    0.0012|
+-------------+-----+----------+
only showing top 20 rows



In [10]:
software_reviews_count = software_reviews.count()
software_helpful_vote = software_reviews.groupBy("helpful_votes").count()
software_helpful_vote = software_helpful_vote.orderBy(software_helpful_vote["count"].desc())
software_helpful_vote.withColumn('percentage', round(software_helpful_vote["count"]/software_reviews_count,4)).show()

+-------------+-----+----------+
|helpful_votes|count|percentage|
+-------------+-----+----------+
|            0| 5054|    0.4853|
|            1| 2315|    0.2223|
|            2| 1069|    0.1026|
|            3|  572|    0.0549|
|            4|  347|    0.0333|
|            5|  223|    0.0214|
|            6|  147|    0.0141|
|            7|  111|    0.0107|
|            8|   65|    0.0062|
|            9|   56|    0.0054|
|           11|   50|    0.0048|
|           10|   41|    0.0039|
|           14|   24|    0.0023|
|           13|   24|    0.0023|
|           12|   24|    0.0023|
|           15|   18|    0.0017|
|           19|   16|    0.0015|
|           16|   15|    0.0014|
|           17|   13|    0.0012|
|           23|   12|    0.0012|
+-------------+-----+----------+
only showing top 20 rows



In [18]:
video_game_star_rating = video_game_reviews.groupBy("star_rating").count()
video_game_star_rating = video_game_star_rating.orderBy(video_game_star_rating["count"].desc())
video_game_star_rating.withColumn('percentage', round(video_game_star_rating["count"]/video_game_reviews_count,4)).show()

+-----------+-----+----------+
|star_rating|count|percentage|
+-----------+-----+----------+
|          4| 1711|    0.3988|
|          5| 1607|    0.3746|
|          3|  718|    0.1674|
|          2|  194|    0.0452|
|          1|   60|     0.014|
+-----------+-----+----------+



In [12]:
software_star_rating = software_reviews.groupBy("star_rating").count()
software_star_rating = software_star_rating.orderBy(software_star_rating["count"].desc())
software_star_rating.withColumn('percentage', round(software_star_rating["count"]/software_reviews_count,4)).show()

+-----------+-----+----------+
|star_rating|count|percentage|
+-----------+-----+----------+
|          4| 3755|    0.3605|
|          5| 3381|    0.3246|
|          3| 2006|    0.1926|
|          2|  769|    0.0738|
|          1|  504|    0.0484|
+-----------+-----+----------+

